In [11]:
import json
import hashlib

In [12]:
def validate_description(description):
    if not description:
        return False
    word_count = len(description.split())
    return 10 <= word_count <= 30

def generate_id(sku):
    # Create consistent ID from SKU
    return int(hashlib.md5(sku.encode()).hexdigest(), 16) % (10 ** 8)

def reformat_zara_item(article):
    if not article.get('structuredData'):
        return None
        
    structured = article['structuredData']
    
    # Skip if description doesn't meet word count criteria
    if not validate_description(structured.get('description', '')):
        return None
        
    reformatted = {
        "ID": generate_id(structured.get('sku', '0')),
        "name": structured.get('name', ''),
        "description": structured.get('description', ''),
        "category": f"Women/{article.get('category', '')}",
        "brand": "Zara",
        "gender": "f",
        "price": str(structured.get('offers', {}).get('price', '')),
        "prodLink": article.get('url', ''),
        "prodImgLink": structured.get('image', '')
    }
    
    return reformatted

In [13]:
mens_source_jsons = ["mens-all-products.json"]
womens_source_jsons = ["women-best-sellers-products.json"]

with open("final-zara-data.json", "w") as final_file:
    processed_jsons = []

    for mens_json in mens_source_jsons:
        with open("mens-all-products.json", "r") as f:
            json_read = json.load(f)
            
            for article in json_read:
                article = reformat_zara_item(article)
                article['gender'] = "m"
                processed_jsons.append(article)

    for womens_json in womens_source_jsons:
        with open(f"womens-best-sellers-products.json", "r") as f:
            json_read = json.load(f)
            
            for article in json_read:
                article = reformat_zara_item(article)
                article['gender'] = "f"
                processed_jsons.append(article)
    
    final_file.write(json.dumps(processed_jsons, indent=4))

TypeError: 'NoneType' object does not support item assignment